In [1]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/home/smoeding2/ca!pip -qqq install langchainches/'
os.environ['XDG_CACHE_HOME'] = '/home/smoeding2/caches/'

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoProcessor,GenerationConfig
from transformers import GenerationConfig, pipeline,TextStreamer
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
import random
import textwrap
from sklearn.metrics import accuracy_score
import pandas as pd

/home/jannik/miniconda3/envs/wsl_pt/lib/python3.11/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
There was a problem when trying to write in your cache folder (/home/smoeding2/ca!pip -qqq install langchainches/). You should set the environment variable TRANSFORMERS_CACHE to a writable directory.
2024-01-13 15:28:44.849167: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-13 15:28:45.059676: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-13 15:28:45.059709: E external/local_xla/xla/stream_

In [3]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3090 Ti'

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = "medalpaca/medalpaca-7b"
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id=tokenizer.eos_token_id




PermissionError: [Errno 13] Permission denied: '/home/smoeding2'

In [ ]:
generation_config=GenerationConfig(max_new_tokens=30,
                                    temperature=0.4,
                                    top_p=0.95,
                                    top_k=40,
                                    repetition_penalty=1.2,
                                    bos_token_id=tokenizer.bos_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
                                    do_sample=True,
                                    use_cache=True,
                                    output_attentions=False,
                                    output_hidden_states=False,
                                    output_scores=False,
                                    remove_invalid_values=True
                                    )

pip install ipywidgets==7.7generation_config=GenerationConfig(max_new_tokens=30,
                                    temperature=0.4,
                                    top_p=0.95,
                                    top_k=40,
                                    repetition_penalty=1.2,
                                    bos_token_id=tokenizer.bos_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
                                    do_sample=True,
                                    use_cache=True,
                                    output_attentions=False,
                                    output_hidden_states=False,
                                    output_scores=False,
                                    remove_invalid_values=True
                                    )# Mistral-7B-v0.1 (foundation model)


In [ ]:
def prompt_model(prompt):
    input_tokens=tokenizer(prompt + " Die richtige Antwort ist BEGINANTWORT", return_tensors="pt").to(model.device)
    output_tokens=model.generate(**input_tokens, generation_config=generation_config, pad_token_id=tokenizer.eos_token_id)[0]
    answer=tokenizer.decode(output_tokens, skip_special_tokens=True)
    return answer

In [ ]:
prompt_case = "Es folgt eine Fallbeschreibung, dessen Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und ENDANTWORT. BEGINFALL <INSERTFALL> ENDFALL BEGINFRAGE <INSERTFRAGE> ENDFRAGE BEGINANTWORT <INSERTANTWORT0> ENDANTWORT BEGINANTWORT <INSERTANTWORT1> ENDANTWORT BEGINANTWORT <INSERTANTWORT2> ENDANTWORT BEGINANTWORT <INSERTANTWORT3> ENDANTWORT BEGINANTWORT <INSERTANTWORT4> ENDANTWORT. Die Frage wird gleich dadurch Beantwortet, die korrekte Antwortmöglichkeit wortwörtlich wiederzugeben."
prompt_no_case = "Es wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und ENDANTWORT. BEGINFALL <INSERTFALL> ENDFALL BEGINFRAGE <INSERTFRAGE> ENDFRAGE BEGINANTWORT <INSERTANTWORT0> ENDANTWORT BEGINANTWORT <INSERTANTWORT1> ENDANTWORT BEGINANTWORT <INSERTANTWORT2> ENDANTWORT BEGINANTWORT <INSERTANTWORT3> ENDANTWORT BEGINANTWORT <INSERTANTWORT4> ENDANTWORT. Die Frage wird gleich dadurch Beantwortet, die korrekte Antwortmöglichkeit wortwörtlich wiederzugeben."

In [ ]:
questions_df = pd.read_csv("leolm/output.csv")
questions_df["Answer LLM"] = [-1 for i in range(len(questions_df))]

In [ ]:
questions_df

In [ ]:
for index, row in questions_df.iterrows():
    if row["Answer LLM"] == -1:
        if type(row["Case"]) == str:
            prompt = prompt_case.replace("<INSERTFALL>",row["Case"])
        else:
            prompt = prompt_no_case
        prompt = prompt.replace("<INSERTFRAGE>",row["Question"])
        for i in range(5):
            prompt = prompt.replace("<INSERTANTWORT" + str(i) + ">",row["Answer "+ str(i+1)].replace(str(i) + ")",""))
        answer = prompt_model(prompt).split("Die richtige Antwort ist BEGINANTWORT")[1].split("ENDANTWORT")[0]
        found = False
        for i in range(5):
            if row["Answer "+ str(i+1)].replace(str(i) + ")","").strip().lower() == answer.strip().lower():
                found = i
                break
        print(answer)
        if found != False:
            questions_df.loc[index, "Answer LLM"] = found
print("Values set:", len(questions_df[questions_df["Answer LLM"] != -1]),"/",len(questions_df))


In [ ]:
questions_df.to_csv("MedAlpaca_output.csv")

In [ ]:
y_pred = questions_df[questions_df["Answer LLM"] != -1]["Answer LLM"]
y_true = questions_df[questions_df["Answer LLM"] != -1]["Correct Answer"]

In [ ]:
print("Accuracy:",accuracy_score(y_true, y_pred))